<a href="https://colab.research.google.com/github/simon-m-mudd/smm_teaching_notebooks/blob/master/lammermuir_data/multiple_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combining multiple point datasets

We are going to assume you have:
* A file with the channel nodes and gradients. This file will be called `killmande_channel_with_gradients.csv`
* A file with channel widths using British National Grid Coordinates
* A file with D50 information, again using British National Grid Coordiantes

Our main data is the D50, so we want to progressively add to that particular dataset

## Import the necessary packages

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point

## Get the files

You need to get a file called `killmade_channel_with_gradient.csv` that is created by the notebook called *Adding_gradient_column_to_csv*

You also need two files, one with widths, and one with D50.

I am going to assume these are called:
* `bankfull_width.csv`
* `grain_size_data.csv`

We first need to look at the data. We will load them in different dataframes.

In [2]:
# Load the channel data
df_channel = pd.read_csv("killmade_channel_with_gradient.csv")

# Load the grain size data
df_grain = pd.read_csv("grain_size_data.csv")

# Load the width data
df_width = pd.read_csv("bankfull_width.csv")

Now we look at what is in each of these files in turn using the `head()` command:

In [4]:
df_channel.head(3)

,Unnamed: 0,latitude,longitude,chi,elevation,flow_distance,drainage_area,source_key,basin_key,geometry,slope,slope_rolling
0,7882,55.839362,-2.555733,8.3116,401.04,4523.8,59624.0,35,4,POINT (-2.55573284 55.839362),0.003419,0.000013
1,7883,55.839344,-2.555701,8.2915,401.03,4520.9,59636.0,35,4,POINT (-2.55570111 55.8393439),0.001694,0.000020
2,7884,55.839326,-2.555669,8.2714,401.03,4518.1,59652.0,35,4,POINT (-2.55566938 55.8393258),0.000000,0.000023


In [5]:
df_grain.head(3)

,Year,D50_mm,D84_mm,x,y
0,2012,86,106,366193,662416
1,2012,79,141,366342,662726
2,2014,47,74,366347,662726


In [6]:
df_width.head()

,Year,Bankfull_m,x,y,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,2012,4.57,366081,661730,NaN,NaN,NaN,NaN
1,2012,4.05,366073,661733,NaN,NaN,NaN,NaN
2,2012,3.01,366054,661728,NaN,NaN,NaN,NaN
3,2012,2.48,366054,661726,NaN,NaN,NaN,NaN
4,2012,2.59,366060,661732,NaN,NaN,NaN,NaN


So, what have we learned here?

1. The channel data has latitude and longitude columns. We can assume the coordinate system is EPSG:4326
2. The other two have x and y columns. These have 6 digit numbers each, and are in the UK. This is British National Grid (EPSG:27700)
3. You can tell UTM from BNG because UTM will have (unless you are near the equator) a 7 digit number for the y (or northing) coordinate.

### Get rid of unnamed data columns

If you processed your data in excel, it is possible you will have a bunch of junk columns called `Unnamed`. We get rid of those this way

In [9]:
# Assuming 'df' is your DataFrame
df_width = df_width.loc[:, ~df_width.columns.str.contains('^Unnamed')]

df_width.head(3)

,Year,Bankfull_m,x,y
0,2012,4.57,366081,661730
1,2012,4.05,366073,661733
2,2012,3.01,366054,661728


## Step 1: Merge channel information onto D50 data

We have two datasets. One is the channel data and the other is data about channel width. This second dataset could be any set of points.

We will, in the next step, merge these datasets based on the nearest neighbour to one of the set of points (i.e., mapping channel data to the nearest channel width point).

For this to work, **the two datasets must be in the same coordinate reference system**.

In the example below, we use `.crs` to define the coordinate reference system. We can do this because we know that one of the datasets is in `EPSG:4326` because it has latitude and longitude data, and the other one is in `EPSG:27700`, which is the British National Grid, because it is mean to mimic data collected by students in the field using GPS that have the British National Grid as default.

We then convert the data from British National Grid to `EPSG:4326` using the function `.to_crs`

In [16]:
# Load the channel data
gdf_channel = gpd.GeoDataFrame(
    df_channel, geometry=gpd.points_from_xy(df_channel.longitude, df_channel.latitude))
# We have to tell the geopandas data what geographic system we are in by using something called an EPSG code.
# All major geographic projection and transformation system have this code.
gdf_channel.crs = "EPSG:4326"


# Load the D50 data
gdf_grain = gpd.GeoDataFrame(
    df_grain, geometry=gpd.points_from_xy(df_grain.x, df_grain.y))
# We have to tell the geopandas data what geographic system we are in by using something called an EPSG code.
# All major geographic projection and transformation system have this code.
gdf_grain.crs = "EPSG:27700"

# IMPORTANT: we convert one of the datasets to the coordinate reference system of the other
gdf_channel_BNG = gdf_channel.to_crs(27700)

The next three lines just show what the first few lines of data looks like.

## Add the function for combining datasets

The below function merges two datasets using nearest neighbours.
**You don't need to change anything in this function.**
The first dataframe keeps its data elements and adds properties from the nearest neighbour that are closest to the points in the first dataframe.

In [13]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ],
        axis=1)

    return gdf

## Merge the two files!

Now we merge the channel widths, that have been converted to the correct coordinate reference system, with the channel data

In [17]:
new_gdp = ckdnearest(gdf_grain, gdf_channel_BNG)
new_gdp.head()

,Year,D50_mm,D84_mm,x,y,geometry,Unnamed: 0,latitude,longitude,chi,elevation,flow_distance,drainage_area,source_key,basin_key,slope,slope_rolling,dist
0,2012,86,106,366193,662416,POINT (366193 662416),8828,55.853741,-2.541608,2.2956,255.87,2240.8,2769900.0,35,4,0.117500,0.027545,1.912177
1,2012,79,141,366342,662726,POINT (366342 662726),8998,55.856535,-2.539210,1.8150,241.13,1836.1,3410200.0,35,4,0.039011,0.024711,1.981880
2,2014,47,74,366347,662726,POINT (366347 662726),8998,55.856535,-2.539210,1.8150,241.13,1836.1,3410200.0,35,4,0.039011,0.024711,3.385092
3,2014,31,59,366022,662067,POINT (366022 662067),8616,55.850605,-2.544232,2.9599,273.73,2737.7,2099400.0,35,4,0.184300,0.049845,2.282670
4,2014,32,56,366073,662249,POINT (366073 662249),8712,55.852238,-2.543414,2.6446,265.43,2512.5,2579800.0,35,4,0.060000,0.028994,3.832729


Super! Now we can print this new dataset to a file using the `.to_csv` function:

In [18]:
new_gdp.to_csv("updated_grain_size.csv")

## Next step, combine the width data with the grain size data

In [19]:
# Load the channel data
gdf_width = gpd.GeoDataFrame(
    df_width, geometry=gpd.points_from_xy(df_width.x, df_width.y))
# We have to tell the geopandas data what geographic system we are in by using something called an EPSG code.
# All major geographic projection and transformation system have this code.
gdf_width.crs = "EPSG:27700"


final_gdp = ckdnearest(new_gdp, gdf_width)

In [20]:
final_gdp.to_csv("final_grain_size.csv")

Okay, lets load this new csv file to see if it has the correct data.

In [21]:
final_gdp.head(3)

,Year,D50_mm,D84_mm,x,y,geometry,Unnamed: 0,latitude,longitude,chi,...,source_key,basin_key,slope,slope_rolling,dist,Year,Bankfull_m,x,y,dist
0,2012,86,106,366193,662416,POINT (366193 662416),8828,55.853741,-2.541608,2.2956,...,35,4,0.117500,0.027545,1.912177,2016,35.1,366190,662409,7.615773
1,2012,79,141,366342,662726,POINT (366342 662726),8998,55.856535,-2.539210,1.8150,...,35,4,0.039011,0.024711,1.981880,2013,2.2,366341,662726,1.000000
2,2014,47,74,366347,662726,POINT (366347 662726),8998,55.856535,-2.539210,1.8150,...,35,4,0.039011,0.024711,3.385092,2016,2.9,366347,662728,2.000000
